# SymPy amplitude model

In [ ]:
# To run in in Google Colab, uncomment the following:

# !pip install expertsystem graphviz

## Generate transitions

As in the doc}`usual workflow <workflow>`, use {func}`.generate_transitions` to create a list of allowed {class}`.StateTransitionGraph` instances for a specific decay channel:

In [ ]:
import expertsystem as es

result = es.generate_transitions(
    initial_state=("J/psi(1S)", [-1, +1]),
    final_state=["gamma", "pi0", "pi0"],
    allowed_intermediate_particles=["f(0)(980)", "f(0)(1500)"],
    allowed_interaction_types=["strong", "EM"],
    formalism_type="canonical-helicity",
)

In [ ]:
import graphviz

graphs = result.collapse_graphs()
dot = es.io.convert_to_dot(graphs)
graphviz.Source(dot)

## Build SymPy expression

Now, instead of using the usual {func}`.generate_amplitudes`, we use {func}`.generate_sympy` from the {mod}`.amplitude` module to formulate the amplitude model in terms of a {mod}`sympy`. The function {func}`.generate_sympy` returns a {class}`.ModelInfo` object that contains a {class}`sympy.Expr <sympy.core.expr.Expr>`:

In [ ]:
model = es.amplitude.generate_sympy(result)
expression = model.expression
expression.top

The {class}`sympy.Expr <sympy.core.expr.Expr>` is currently just a sum of {class}`sympy.Symbol <sympy.core.symbol.Symbol>` instances that represent a sum of intensities. You can now recursively replace ({meth}`~sympy.core.basic.Basic.subs`) these symbols with {attr}`.SympyModel.intensities` and {attr}`.SympyModel.amplitudes`:

In [ ]:
expression.top.subs(expression.intensities)

{class}`.ModelInfo` also contains a mapping of {attr}`.ModelInfo.parameters` to some suggested parameter values ({class}`.ParameterProperties`). These can be used to substitute symbols in the expression with initial values:

In [ ]:
import sympy as sy

initial_values = {k: v.value for k, v in model.parameters.items()}
evaluated_wigner_d = sy.expand(
    expression.full_expression.subs(initial_values).doit()
)
evaluated_wigner_d

Note: we use {meth}`~sympy.core.basic.Basic.doit` to evaluate the Wigner-$D$ ({meth}`Rotation.D <sympy.physics.quantum.spin.Rotation.D>`) and or Clebsch-Gordan ({class}`~sympy.physics.quantum.cg.CG`) functions in the full expression.

## Visualize

In this case ($J/\psi \to \gamma f_0, f_0 \to \pi^0\pi^0$) _without dynamics_, the total intensity is only dependent on the $\theta$ angle between the two $\pi^0$'s:

In [ ]:
import sympy as sy

theta = next(iter(evaluated_wigner_d.free_symbols))
sy.plot(
    evaluated_wigner_d,
    (theta, 0, sy.pi),
    axis_center=(0, 0),
    ylabel="$I$",
    ylim=(0, 16),
);

This intensity is built up of four components:

In [ ]:
display(*expression.intensities)

In [ ]:
evaluated_wigner_d.free_symbols

This can be nicely visualized as follows:

In [ ]:
import sympy as sy

plots = list()
colors = ["red", "blue", "green", "purple"]

total = 0
for i, (label, intensity) in enumerate(expression.intensities.items()):
    total += (
        intensity.subs(expression.amplitudes)
        .subs(expression.dynamics)
        .subs(initial_values)
        .doit()
    )
    plots.append(
        sy.plot(
            total,
            (theta, 0, sy.pi),
            axis_center=(0, 0),
            ylabel="$I$",
            ylim=(0, 16),
            line_color=colors[i],
            show=False,
            label=f"${label.name}$",
            legend=True,
        )
    )
for i in range(1, 4):
    plots[0].extend(plots[i])
plots[0].show()

## Set dynamics

Note that the {class}`.ModelInfo` class contains a few dynamics symbols. By default, these dynamics symbols are substituted to "non-dynamic" (equal to $1$).

In [ ]:
expression.dynamics

You can set the values of this {attr}`~.SympyModel.dynamics` mapping to be any kind of {class}`~sympy.core.expr.Expr`, as long as you keep track of which {class}`~sympy.core.symbol.Symbol` names you use. The {mod}`expertsystem` does provide a few common {mod}`.lineshape` functions however, which can be constructed as {class}`~sympy.core.expr.Expr` with the correct {class}`~sympy.core.symbol.Symbol` names using {func}`.set_resonance_dynamics`. This function takes specific {mod}`.builder` functions, such as {func}`.create_relativistic_breit_wigner`, which would create a {func}`.relativistic_breit_wigner` for a specific resonance. Here's an example for a relativistic Breit-Wigner _with form factor_:

In [ ]:
from expertsystem.amplitude.dynamics import set_resonance_dynamics
from expertsystem.amplitude.dynamics.builder import (
    create_relativistic_breit_wigner_with_ff,
)

set_resonance_dynamics(
    model, "f(0)(980)", create_relativistic_breit_wigner_with_ff
)
set_resonance_dynamics(
    model, "f(0)(1500)", create_relativistic_breit_wigner_with_ff
)
assert len(expression.dynamics) == 4

## Plot the model

At this stage, we have several free symbols. For the fitting package these will be considered **parameters** that are to be optimized and (kinematic) **variables** that represent the data set. Examples of parameters are mass ($m_\text{particle}$) and width ($\Gamma_\text{particle}$) of the resonances and certain amplitude coefficients ($C$). Examples of kinematic variables are the helicity angles $\theta$ and $\phi$ and the invariant masses ($m_{ij...}$).

In [ ]:
display(*expression.full_expression.free_symbols)

Let's say we want to plot the amplitude model with respect to $m_{3+4}$. We would have to substitute all other free symbols with some value.

First, we can use {attr}`.ModelInfo.parameters` to substitute the parameters with suggested values:

In [ ]:
suggested_expression = expression.full_expression.subs(
    {k: v.value for k, v in model.parameters.items()}
)
suggested_expression.free_symbols

Ideally, we would now 'integrate out' the helicity angles. Here, we however just set these angles to $0$, as computing the integral would take quite some time:

In [ ]:
angle = 0
angle_substitutions = {
    s: angle
    for s in suggested_expression.free_symbols
    if s.name.startswith("phi") or s.name.startswith("theta")
}
evaluated_angle_intensity = suggested_expression.subs(angle_substitutions)
evaluated_angle_intensity.free_symbols

By now we are only left with the masses of the final state particles ($m_3$ and $m_4$), since they appear as symbols in the {func}`.relativistic_breit_wigner_with_ff`. Final state particles 3 and 4 are the $\pi^0$'s, so we can just substitute them with their masses.

In [ ]:
from expertsystem.io import load_pdg

pi0 = load_pdg()["pi0"]
plotted_intensity = evaluated_angle_intensity.doit().subs(
    {
        sy.Symbol("m_3", real=True): pi0.mass,
        sy.Symbol("m_4", real=True): pi0.mass,
    },
    simultaneous=True,
)

```{note}
Use {meth}`~sympy.core.basic.Basic.subs` with `simultaneous=True`, as that avoids a bug later on when plotting with {mod}`matplotlib.pyplot`.
```

That's it! Now we are only left with the invariant mass $m_{3+4}$ of the two pions:

In [ ]:
assert len(plotted_intensity.free_symbols) == 1
m = next(iter(plotted_intensity.free_symbols))
m

...and we can plot it with with {func}`sympy.plot <sympy.plotting.plot.plot>`:

In [ ]:
sy.plot(
    plotted_intensity,
    (m, 2 * pi0.mass, 2.5),
    axis_center=(2 * pi0.mass, 0),
    xlabel=f"$m(\pi^{0}\pi^{0})$",
    ylabel="$I$",
    backend="matplotlib",
);

The expression itself looks like this (after some rounding of the `float` values in this expression:

In [ ]:
from sympy import preorder_traversal

rounded_intensity = plotted_intensity
rounded_intensity = rounded_intensity.subs({sy.sqrt(10): sy.sqrt(10).n()})
for a in preorder_traversal(rounded_intensity):
    if isinstance(a, sy.Float):
        rounded_intensity = rounded_intensity.subs(a, round(a, 2))
rounded_intensity

Finally, here's a small snippet of how to use {func}`~sympy.integrals.integrals.integrate`. Again, it's not evaluated here, because that takes too much time:

In [ ]:
# sy.integrate(
#     rounded_intensity,
#     (m, 2.01 * pi0.mass, sy.oo),
#     meijerg=True,
#     conds="piecewise",
#     risch=None,
#     heurisch=None,
#     manual=None,
# ).evalf(5)